**Example of MNIST** 
---

The data and results will be stored in the following directories
1. drive/My Drive/public/data/ has data
2. drive/My Drive/public/results/ will have results


For testing the trained model with arbitrary images, copy the image files into drive/My Drive/public/data/mnist_test_images/. images are available at 

https://drive.google.com/open?id=1zIcByY9Jg2byx_R0oWEqRPvLDEUfo9-c 


Mount your good drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


Import pytorch

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Define your network model. We have two models. 

In [ ]:
#@title

"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 300)
        self.fc3 = nn.Linear(300, 100)
        self.fc4 = nn.Linear(100, 10)

    def forward(self, x):
        batch_size, c, h, w = x.data.size()
        x = x.view(batch_size, 784)
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)
"""

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 20, 5, 1), # out: (20, 24, 24)
            nn.ReLU(),
            nn.MaxPool2d(2,2), # out: (20, 12, 12)
            nn.Conv2d(20, 50, 5, 1),# out: (50, 8, 8)
            nn.ReLU(),
            nn.MaxPool2d(2,2) # out: (50, 4, 4)
        )
        conv_size = self.get_conv_size((1, 28, 28)) # obtain output size of conv
        self.fc = nn.Sequential(
            nn.Linear(conv_size, 500), # conv_size = 4*4*50
            nn.Linear(500, 10)
        )
    
    def get_conv_size(self, shape): 
        o = self.conv(torch.zeros(1, *shape)) # forward through conv layer
        return int(np.prod(o.size())) # outputs the # of output

    def forward(self, x): # 
        batch_size, c, h, w = x.data.size() # 32*1*28*28
        x = self.conv(x)
        x = x.view(batch_size, -1) # conv_size = 4*4*50
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [ ]:
mydata = (1,2,3)
torch.zeros(1, *mydata)

tensor([[[[0., 0., 0.],
          [0., 0., 0.]]]])

definition of train and test functions

In [ ]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # data: (bsz 1 28 28) target : (bsz)
        optimizer.zero_grad()
        output = model(data) # output: (bsz, 10)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

            #torch.save(model.state_dict(),"drive/My Drive/public/results/mnist_cnn.pt")


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            #TODO: sent data and target to device and feed to the model#
            data, target = data.to(device), target.to(device)
            output = model(data)
            ############################################################

            # sum up batch loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()  

            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True) # pred: (bsz 1)
            correct += pred.eq(target.view_as(pred)).sum().item() # (bsz 1) vs. (bsz 1) / element-wise equality

    test_loss /= len(test_loader.dataset) # n_samples

    print('\nTest: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
a = torch.tensor([1,2,3]).view(-1,1)
print(a)
b = torch.tensor([2,2,3])
b= b.view_as(a)
print(b)
correct = a.eq(b)
correct

---
main code 

In [ ]:
seed = 1
epochs = 2
batch_size = 32
test_batch_size = 1000
lr = 0.001
momentum = 0.9
log_interval = 100
save_model = True

torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {} # pin_memory: speed up transfer from cpu to gpu
transform=transforms.Compose([
   transforms.ToTensor(),
   transforms.Normalize((0.1307,), (0.3081,)) ]) # mean and std for each channel
train_loader = torch.utils.data.DataLoader( # load trainset
    datasets.MNIST('drive/My Drive/public/data', train=True, 
                   download=True, transform=transform),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader( # load testset
    datasets.MNIST('drive/My Drive/public/data', train=False, 
                   transform=transform),
    batch_size=test_batch_size, shuffle=True, **kwargs)



In [ ]:
for batch, (data, target) in enumerate(train_loader):
  print(data, target)
  break


In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

if (save_model):
    if not os.path.exists('drive/My Drive/public/results'):
        os.mkdir('drive/My Drive/public/results')
    torch.save(model,"drive/My Drive/public/results/mnist_cnn.pth")

In [ ]:
!ls "drive/My Drive/public/results"

mnist_cnn.pth


After Training
* load the trained model
* test with one image or with one directory

In [ ]:
load_model = torch.load("drive/My Drive/public/results/mnist_cnn.pth")

NameError: ignored

In [ ]:
from skimage import io

img_name = 'drive/My Drive/public/data/mnist_test_images/test_0.jpg'
test_img = io.imread(img_name).reshape(28,28)
test_data = transform(test_img).view(1,1,28,28).to(device)

with torch.no_grad():
  output=load_model(test_data)
print(img_name, output.argmax(dim=1).cpu().numpy()[0])

FileNotFoundError: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.title(img_name.split('/')[-1])
plt.imshow(test_img, cmap='gray')
plt.show()
plt.clf()

NameError: ignored

In [ ]:
from skimage import io
import glob

file_list = glob.glob("drive/My Drive/public/data/mnist_test_images/*.*")
for img_name in file_list:
  test_img = io.imread(img_name).reshape(28,28)
  test_data = transform(test_img).view(1,1,28,28).to(device) # transform processes the image easily
  
  plt.title(img_name.split('/')[-1])
  plt.imshow(test_img, cmap='gray')
  plt.show()
  plt.clf()

  with torch.no_grad():
    output=load_model(test_data)
  print(img_name, output.argmax(dim=1).cpu().numpy()[0])
  

the end! 
----

# New Section